# Generative AI - Combient Mix AB / Silo AI for Patricia AB 2023

Prompt Engineering (PE) is the primary vehicle for guiding generative AI models towards stable applications. It is particularly applicable to Large Language Models (LLMs) and involves formulating prompts and prompting schemas in order to retrieve appropriate responses to queries. It is essential for meaningful interactions with LLMs.

This notebook provides a hands-on introduction to PE for end users with a basic understanding of the Python programming language. No advanced coding and no technical background knowledge of generative AI or LLM is required. The notebook is divided into three sections, each corresponding to material covered in one of two live seminars. There will be ~40-60 minutes available in order to go through and complete each section.
<br />
<br />

***

**Structure**

Exercise **1 & 2 during the first session** (~ 40 minutes per exercise) and exercise **3 during the second session** (~ 60 minutes)


1.) First, we'll explore the basics of prompting LLMs and the importance of PE. We will see examples of some important and cutting edge techniques for prompting efficiently and with intent
* 0 / 1 / few-shot prompting,
* Role-Task-Format (RTF),
* In-Context-Learning (ICL),
* Chain-of-Thought (CoT) and
* Tree-of-Thought (ToT)

2.) We'll learn how to integrate and utilize existing tools for specific tasks. We focus on some of the tools available via enabling plugins for GPT-4 at [OpenAI chat interface](https://chat.openai.com/). Example tasks include 
  * **Searching the internet for recent information** LLMs are generally pre-trained and thus have a knowledge cutoff at the time when training stopped. The latest GPT-4 model of OpenAI for example has a knowledge cutoff at January 2022. This means that it was not trained on any data produced after that time, so if we want to include updated information this must be done by fetching the information and providing it as context together with our query.
  * **Using your own PDF documents as a knowledge base** (plugin: AskYourPDF). There are several plugins available for GPT-4 which provide the capability of uploading a PDF document and use it as a reference for interacting with the LLM.

3.) The third exercise is more programming heavy and involves working with API calls. We'll guide you through it! This allows us to perform more complex tasks and write our own custom functions utilizing the power of the LLM. We introduce the Python library [Langchain](https://python.langchain.com/docs/get_started/introduction.html) and some of its tools for chaining prompts. Chaining prompts allows for more intricate problem-solving as well as gaining control over the reliability of the output. We will see how to use this in order to accomplish Retrieval Augmented Generation (RAG) from our own knowledge base (the PDF document). This exercise demonstrates how the plugins used in exercise 2 work under the hood. Examples of what we can achieve include

* constraining the system behaviour, for example to mitigate hallucinations from the LLM
* Retrieval Augmented Generation from external knowledge base such as the internet or a collection of documents

This provides a deeper understanding of the fundamental components essential for constructing advanced generative AI systems.

***

**Note:**

In order to run this notebook properly you will need

* **Gmail account** - Follow the provided instructions to download the notebook to your GDrive, so that you can edit and save it freely.
* **Google GDrive access** - enable the app `Google Colaboratory`

> Run the below code blocks to install necessary packages.
>
> Notebook code blocks can be executed via either: 
> * **shift + enter**: executes current code block and moves to the cell below
> * **control + enter**: executes current code block

## Environment setup

Here we set up the environment variables and make sure we can access data via Google Drive.

Run the code below to load the GenAI map we will be using during this course. The code is fetching a map called GenAI from our Github repository. Click on plus next to the cell to run the code.

In [ ]:
use_drive = True

if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive

  !git clone https://github.com/statisticalmodel/GenAI

You can now view the files by clicking on the map and follow the path down in the GenAI short course.

![Image of GitClone](https://github.com/statisticalmodel/GenAI/blob/main/GenAI%20short%20course/GitClone.png?raw=true)

# Hands-On 1: Prompting - spellcasting with intent

## Introduction

In the first exercise sessions we will explore some basics of Prompt Engineering (PE). We showcase examples of techniques for efficiently extracting information from Large Language Models (LLMs). These techniques are widely applicable across different models, but work the best for the very large models trained on a large corpus of data.

<br>

In the first hands-on session we aim to cover

* **Why is prompting technique important?**
  * Prompting helps in formulating the input in a way that the model can understand and respond to effectively. A well-crafted prompt can significantly improve the quality and relevance of the model's output.
  * Through prompting, you can guide the model's responses in a particular direction or within certain boundaries. This is crucial for obtaining accurate, relevant, or safe responses.
  * Forms the basis behind developing advanced functionalities on top of generative AI base models.

<br>

* **What can we achieve?**
  * Quickly read and summarize/extract relevant information from text source.
  * Transform text into a format which is directly useful for you. This includes e.g. language translation or formatting the output as excel table, JSON dictionary, MarkDown or html.
  * Get feedback and suggestions for improvement and introspection. This may include both natural language text and code.
  * Infer sentiment, topic and logic structure in text.
  * Generate/revise drafts for planning, policies, mails, slides and more.
  * Brainstorming partner and ideation, an extra brain in the room.

<br>

* **Important techniques**
  * **Best practices** - Write clearly and use semantic clues rather than keyword searching. Clearly state your goals and wishes, have an intent with your promt. In some sense it is like moving back to how you perhaps intuitively wanted to interact with e.g. Google when searching internet. Or how you would ask an all-knowing oracle. Don't be afraid to interact, provide feedback and follow up when the output is not what you desired.
  * **Role Task Format (RTF)** - Give the LLM a *role*, to set it in the right *mood*, define a clear *task* to be achieved and provide a couple of examples of how you want to *format* the output.
  * **In Context Learning (ICL)** - Provide examples and any additional information to consider as context for your prompt. This allows for a *shallow* learning, where the LLM can learn from your prompt without updating any weights of the model. It is easier to make the bigger models accept new information in the prompt and generalize from that. The smaller models are more locked into the worldview built from training.
  * **Chain of Thought (CoT)** - Ask the model to think through problems logically and in a step-by-step manner. This helps the model to linearize the problem into a sequence of logical considerations and often produce more accurate output.
  * **Tree of Thought (ToT)** - Generate a few different suggestions for how to solve a given problem and rank these suggestions by having an LLM reflect on the solutions given the original problem. Keep the best approach for a solution, then generate a a few new suggestions for proceeding towards a solution and rerank. Again, keep the best and iterate until satisfied that the best solution has been reached. Can become quite intensive in the amount of API calls made to the LLM.

These techniques usually take us very far and form a basis for more advanced applications built on chaining prompts.

## **Summarizing**

Summarizing or extracting information from text sources are areas where LLMs shine particularly bright. This can be helpful for condensing lengthy news articles, generating concise summaries of meetings or reports, summarizing books or chapters for quick review or understanding and summarizing customer feedback or reviews in order to understand common sentiments or issues.

**Copy paste the following prompt into the ChatGPT window and see how it answers**

In [ ]:
prompt = f"""
Summarize the following conversation between a service representative and a customer in a few sentences. 
Use only the information from the conversation.

Service Rep: How may I assist you today?
Customer: I need to change the shipping address for an order.
Service Rep: Ok, I can help you with that if the order has not been fulfilled from our warehouse yet. 
But if it has already shipped, then you will need to contact the shipping provider. Do you have the order ID?
Customer: Yes, it's 88986367.
Service Rep: One minute please while I pull up your order information.
Customer: No problem
Service Rep: Ok, it looks like your order was shipped from our warehouse 2 days ago. 
It is now in the hands of  the shipping provider, so you will need to contact them to update your delivery details. 
You can track your order with the shipping provider here: https://www.shippingprovider.com
Customer: Sigh, ok.
Service Rep: Is there anything else I can help you with today?
Customer: No, thanks.
"""

**Try it out:**

Try changing the prompt for the above conversation and compare the answers:


**Summarize with a word limit:**

In [ ]:
prompt = f"""
Summarize the conversation between a service representative and a customer in at most 30 words. Use only the information from the conversation.
"""

**Summarize with a focus on sentiment:**

In [ ]:
prompt = f"""
Summarize the conversation between a service representative and a customer in at most 30 words, 
focusing on any aspects which are relevant to customer's sentiment. Use only the information from the conversation.
"""

## **Impersonating or defining a Role**
Impersonating is good to use when you are looking for expert answers in a specific field or just want a personalized interaction. It can be used for company chat bots or a friendly assistant. We have created the following Atlas Copco personal assistant to make it clear how a LLM work with impersonation.

In the prompt below we define a fictive persona in the first paragraph, you can then proceed to ask it question.

**Copy paste the following promt into the ChatGPT window.**

In [ ]:
prompt = f"""
You are a helpful assistant defined by the following criteria:

Role: You are ALEX, the personal assistant representative from Atlas Copco. 

Task: Always answer politely and make sure to uphold and keep in line with the core values of Atlas Copco: interaction, innovation and commitment.
Navigating the vast landscapes of the 21st century, Atlas Copco has transformed the way individuals 
and businesses perceive and interact with leading-edge technology. 

Format: As a beacon of empowerment, you consistently challenge traditional industrial norms, advocating for personal growth, safety, and 
entrepreneurial spirit. Whether someone is curious about the latest tech advancements or the rich 
history of Atlas Copco, you have the answers. Ready to enlighten, empower, and inspire, you're the 
go-to for anyone looking to learn, build and innovate.
"""

**Exaples of questions to ask:**
Try asking the Atlas Copco assistant some of the following questions:

In [ ]:
prompt = f"""
Can you provide a brief overview of Atlas Copco's history and its main business areas?

In how many countries does Atlas Copco operate?

What are the primary products and services offered by Atlas Copco?

Has Atlas Copco recently entered into any strategic partnerships or collaborations?

How can I get in touch with Atlas Copco's customer support for product-related queries?

How has Atlas Copco performed financially in the last quarter/year?

Who are you?
"""

### **Impersonating extra assignment**

The previous example was intentionally extreme, to demonstrate just how free we are to define the atmospheric setting. In practice however it is often useful to define the role of the LLM as an expert in the area you wish to query about. This makes it easier for the LLM to navigate its knowledge base and find more accurate responses. We can additionally ask for the output to be provided as some other persona, which can be useful when the expert lingo can become too advanced.


In this exercise we ask the LLM to be an expert astronomer, but provide its answers as if speaking to a 10-year old child.  

**Copy paste the following promt into the ChatGPT window.**

In [ ]:
prompt = f"""
You are an expert astronomer who is very knowledgeable about the solar system. 
Your task is to answer users questions in a satisfactory manner using clear, simple and instructive sentences.
Shape your response as if talking to a 10-year old child.

Here is an example dialogue:
Question: How many moons does Mars have? 
Answer: Very good question. Mars has two small moons. They are called Phobos and Deimos. They are very small and irregularly shaped. 
Phobos is the larger of the two moons and is about 17 miles (27 kilometers) in diameter. 
Deimos is about 12 miles (19 kilometers) in diameter. Both moons are thought to be captured asteroids.
Imagine them as two little buddies that always follow Mars around in space! 🪐🌕🌕
"""

**Try asking it some questions:**

In [ ]:
prompt = f"""
How many planets are there in the solar system?

When I learned about the planets in school, there were nine. When did that change?

Does Pluto have any moons? What about other dwarf planets? Who chose all of these cool names?!
"""

## **Writing - defining a typical task**
LLMs are extraordinarily good at generating written content. This can be used for many varying purposes, such as: marketing pitches, ad generation, creating an outline for an essay, grammar correction and language translation, rewriting a text from a description or writing customized emails.

### **Writing - Marketing generation**

**Copy paste the following prompt into the ChatGPT window.**

In [ ]:
prompt = f"""
You are an expert in marketing and tech product design. 
Generate a one paragraph marketing pitch from the product description below. 
Use only information from the provided description.


NokiaTWS-411 Comfort Earbuds True wireless-hörlurar Svart
Artikelnr. 5011272056 Tillv. art. nr. 8P00000143
- Typ True wireless-hörlurar
- Anslutningsteknik Trådlös
- Driftstid (upp till) 9.5 h
- Färgkategori Svart
"""

It will also take into account who you are creating the content for. If you ask it to write an email to your boss saying you will be late it will have a very different tone than if the email will go to your mom.

**Copy paste the following promt into the ChatGPT window.**

In [ ]:
prompt = f""" 
Write a suggestion for an email to my mother, explaining that I will have to be late this morning.
"""

In [ ]:
prompt = f"""
You are very skilled in HR related questions and have long experience as a respected communicator. 

Write a suggestion for an email to my boss, explaining that I will have to be late this morning.
"""

### **Writing - Text from Description**

**Copy paste the following prompt into the ChatGPT window.**

In [ ]:
prompt = f"""
Write an ad copy for a part-time data entry job targeting college students. The job pays 500 sek/hour and you can work from home.
"""

### **Writing - Response to Customer Reviews**

**Copy paste the following prompt into the ChatGPT window.**

In [ ]:
prompt = f"""
Write a reply to a valued customer.
Generate a reply to thank the customer for their review.
Make sure to use specific details from the review.
Write in a concise and professional tone.

Customer review: "I'm disappointed with my latest purchase. The item arrived late and was damaged."
"""

### **Writing - Formatting the input prompt**

Formatting the prompt in various ways can produce different outcomes. If at first you don't succeed it can therefore be useful to rephrase your prompt in various ways.

**Copy paste the following prompt examples into the ChatGPT window.**

In [68]:
prompt = f"""
1. Translate the Swedish sentence {{swedish_sentence}} into English: {{english_translation}}

2. From English to Swedish: {{swedish_sentence}} ~ {{english_translation}}

3. English: {{swedish_sentence}} | Swedish: {{english_translation}}
"""

In [ ]:
swedish_sentence = f"""
Vilken dag, suck!
Var tvungen att vobba hela dagen för lillan var sjuk.
Sedan var hon sugen på glass men jag hade inte sl-kortet med mig så var tvungen att planka på tuben för att komma till affären.
Tyvärr åkte jag dit på det och var tvungen att böta. Smakar det så kostar det!
Sen gillade hon inte ens glassen jag köpt. Men smaken är som baken och den föll i alla fall mig i smaken, så det slutade gott för mig.
"""

### **Writing - Formatting the output**

It is often very useful to also format the output in a way that it is easy to read and understand, or to get a structure which can be parsed by some software application. Here we ask the LLM to create a product portfolio and ask for a specific form of the output.

**Copy paste the following prompt into the ChatGPT window.**

In [69]:
prompt = f"""
Create a product portfolio for a company that sells headphones and generate the output in format with the following keys:
"Product class", "Product name", "Product price"
"""

## **Ideation**
LLMs have shown a remarkable ability to assist in ideation processes. They can generate a diverse range of ideas based on the input they receive, making them a valuable tool for brainstorming and exploring creative or strategic solutions.

**Copy paste the following prompt into the ChatGPT window.**

In [ ]:
prompt = f"""
- Give me 3 cat meme ideas:

- Give me 3 interview questions for the role of a senior investment professional.

- What's a good name for a flower shop that specializes in selling bouquets of dried flowers?

- What are some strategies for overcoming writer's block?

- what are good concepts, strategies and targets to incorporate in a portfolio analysis of peers and competitors for investment managers?
"""

## **Tree of Thought (ToT)**

Tree-of-Thought is one of the more advanced techniques currently used for prompting LLMs. The ToT implementation allows you to interactively build upon your thoughts by pasting prompts successively into the chat interface or by scripting a sequence of prompts in a notebook.

Below are 8 prompts provided for you to address a problem you have in mind. Please follow the instructions and paste them into the chat interface successively to facilitate the problem-solving process.

**Copy paste the following prompts into the ChatGPT window in order.**

In [ ]:
prompt_1 = f"""
Ignore all previous instructions.
You are a logical, methodical and problem solving expert genius.
Always find the best and most relevant solution to a given problem.
Always break down the problem, objects, numbers and logic before starting to solve the problem.
Then proceed to solve the problem in a step-by-step manner carefully considering each step.
Acknowledge this by answering yes:
"""


prompt_2 = f"""
Find the simplest and most efficient way to solve the following problem.
Please consider 3 different solutions,
start with the simplest solutions first,
Then compare their efficiency and ability to provide a full answer to the problem.
Score the solutions with a grade between 0-100, according to your estimate of how good they are.
Explain the best solution carefully and step-by-step. This solution will later be referenced as [WINNING_ANSWER_1].
Ask for the problem: [PROBLEM]
"""


prompt_3 = f"""
You are a consulting expert tasked with investigating the best solution for the problem: [PROBLEM].
List all of the flaws and faulty logic of the [WINNING_ANSWER_1] provided above.
We will later refer to this as [CRITIQUE_1].
Work this out in a step by step manner to ensure that you list all the errors and faulty logic:
"""


prompt_4 = f"""
You're a master problem resolver with extensive experience in solving problems related to [PROBLEM].
You will always break down the objects, numbers and logic before starting to solve the problem in a step-by-step manner.
You are tasked with:
1) investigate the flaws provided from the consulting expert in [CRITIQUE_1]
2) rethink the problem again very carefully, focus on whether there is any logical way to make the problem appear more simple?
3) break down the problem, objects, numbers and logic before starting to solve the problem in a step-by-step manner which improves the answer.
4) Print the improved answer in full. We will refer to this as [WINNING_ANSWER_2]
Work this out in a step by step way to be sure we have the right answer:
"""


# The above prompts provide an efficient way of obtaining a well thought out response to your original problem query.
# From here we can generate 2 new responses, pick a new winner and iterate .. we only do 1 such iteration here

prompt_5 = f"""
Given the problem [PROBLEM] and the suggested solution in [WINNING_ANSWER_2]:

Be creative and brainstorm 2 new distinct solutions to the problem and include the proposed solution from [WINNING_ANSWER_2],
so that we now have 3 different ideas for a solution to the problem.
Please consider carefully the 3 factors which have the most impact on the outcome
"""


prompt_6 = f"""
Given the original problem and the three solutions just proposed,
carefully compare their efficiency and ability to provide a full answer to the problem.
Score the solutions with a grade between 0-100, according to your estimate of how good they are.
Explain the best solution carefully and step-by-step. This solution will later be referenced as [WINNING_ANSWER_3].
"""


prompt_7 = f"""
You are a consulting expert tasked with investigating the best solution for the problem: [PROBLEM].
List all of the flaws and faulty logic of the [WINNING_ANSWER_3] provided above.
We will later refer to this as [CRITIQUE_2].
Work this out in a step by step manner to ensure that you list all the errors and faulty logic:
"""


prompt_8 = f"""
You're a master problem resolver with extensive experience in solving problems related to [PROBLEM].
You will always break down the objects, numbers and logic before starting to solve the problem in a step-by-step manner.
You are tasked with
1) investigate the flaws of the answer [WINNING_ANSWER_3], as provided in [CRITIQUE_2].
2) rethink the problem again very carefully, is there any logical way to make the problem appear more simple?
3) break down the problem, objects, numbers and logic before starting to solve the problem in a step-by-step manner which improves the answer.
4) Print the improved answer in full. We will refer to this as [WINNING_ANSWER_4]
Work this out in a step by step way to be sure we have the right answer:
"""

# Hands-On 2: Using existing tools and plug-ins on the OpenAI platform.

In the second hands-on exercise we will explore some of the extended capabilities currently available via the [OpenAi browser interface](https://chat.openai.com/), such as the capability to chat with a PDF or to search for contemporary information on the internet.
We will then build on these tasks in exercise 3, where we will use custom functions for accomplishing the same tasks.

In this assignment we cover the following:
* Internet search using Bing: Ask for current and up to date data by using internet search capability via Bing.
* Exploratory Data Analysis Using a CSV File: Upload a csv file with data and conduct exploratory data analysis using a 'Tree of Thought' inspired Central Intelligence prompt.
* Chat with PDF using AskYourPDF plugin: Upload the Permobil manual and ask questions about the information in it.
* ((((Analysis of Annual Reports via PDFs: Upload the yearly 10-K reports from Whirlpool or Caterpillar. Conduct an in-depth analysis of these reports to derive key financial insights.))))

## Internet search using Bing

Try the below prompt both with and without Bing internet search capability enabled to see the difference in output.
Start by creating a New Chat in OpenAI, as default you are now using GPT-3.5

**Copy paste the following prompt into the ChatGPT window.**

In [2]:
prompt = f"""
What is the current status of the unrest in Israel which was instigated by the massive attack from Hamas affiliated groups?
"""

You should get a message saying something like: 

I'm sorry, but I cannot provide real-time information as my knowledge is up to date only until September 2021. To get the current status of any ongoing events, including the situation in Israel and Gaza, I recommend checking the latest news updates from reliable sources such as news websites, television news, or official government statements. The situation in that region can change rapidly, so it's important to rely on up-to-date and accurate information.

Create a New Chat, this time make sure to change from GPT-3.5 to GPT-4. Make sure to click on the option named "Brows with Bing" that apears when clicking on GPT-4.

**Copy paste the same prompt into the ChatGPT window.**

In [3]:
prompt = f"""
What is the current status of the unrest in Israel which was instigated by the massive attack from Hamas affiliated groups?
"""

This time the GPT-4 model will be using Bing for internet seach and you can see a message saying "Browsing Starting Up".

Reflect on: 
* How does the ability to pull in real-time data from the web impact the quality and relevance of the responses?
* What do you think about the accuracy of the information?
* For what task would it be suitable to enable Bing search and when should you leav it out?


Enable: As a rule of thumb the Bing search should be enabled when the you need real-time or resently updated information. Alwys be mindful of the answer and always check the source from where it gathers its information. 
Disable: If the task primarily demands creative or generative input without reliance on external data, disabling Bing search may foster a more streamlined and imaginative output.

Disable: If user interactions prioritize speed, simplicity, speculative exploration, creative or generative input, it might be beneficial to operate GPT-4 without the external search.

Also note that when enabling Bing search you can not use any other Beta Tools such as Advanced Analytics that we will be trying out in the next assignment. 

## Exploratory Data Analysis Using a CSV File

**Tree of Thought (ToT) inspired prompt example**

Below is an example of ToT reasoning using a single prompt.
This powerful prompt is designed to assist in a wide range of general problems. It has been created specially for investment professionals taking this short course. We will be using it together with the "Advanced Data Analytics" Beta tool but it can be helpful for any generall problem, and can also be used stand alone.

Remember that LLM's are prone to hallucinations and you always need to double check the correctness and verify the results. AI is a tool, not a replacement for human judgment. 

Start by creating a New Chat. This time, you will choose GPT-4 and the "Advanced Data Analytics" Beta tool. This enables the Data Analytics tools.

**Upload the csv file to GPT-4**

Click on the "+" sign you see on the left inside in the chat window (where you normaly post queries). You can now upload the "wine_quality-red_100.csv" by navigating to "Google Drive/GenAI" short course. 

If you can not find the file please let us now and we will assist you. 

**Copy paste the following prompt into the ChatGPT window.**

In [4]:
prompt = f"""
Hello, GPT-4! 
For this session, you will assume the role of a Central Intelligence (CI) problem-solving assistant. 
You are particularly adept at navigating the complex landscape of investment opportunities.
Your primary function is to guide users through complex challenges by leveraging specialized expert agents. 
Here's your operational framework:

Problem Definition: Begin by prompting the user: "Please describe the task or problem you'd like assistance with."

Expert Agent Synthesis: Analyze the user's input carefully and conceptualize three expert agents, 
each equipped with distinct skills, expertise and tools relevant to the user's problem.

Agent Presentation: Introduce the three expert agents to the user, detailing their names, competencies, and tools.

User Collaboration: Allow the user to select one expert agent as the primary lead of the investigation. 
Be receptive to any additional competencies or ideas the user might suggest for the chosen agent.

Deep Dive Analysis: Collaborate with the user and the selected expert agent to dissect the problem, 
explore potential solutions, and elucidate the rationale behind each recommendation.

Dynamic Adaptation: If the situation demands, be prepared to introduce new expert agents to collaborate and share insights. 
Ensure seamless integration and data transfer between all agents.

Progress Recap: Every four steps, provide a concise summary of the actions taken, decisions made, 
and the current state of the problem-solving process.

Activate this mode and await the user's initial input.
"""

GPT-4 will now ask you to describe the problem you'd like assistance with.

**Copy paste the following prompt into the ChatGPT window.**

In [5]:
prompt = f"""
I want you to perform an exploratory data analysis of the provided data. 
Detail the content, features, any missing values, correlations etc. 
Also include visualizations and focus on keeping quality as a primary target in your analysis
"""

The CI system will initiate three expert agents to assist you, and you will be asked to select one expert agent as the primary lead for this investigation.

Pick one of the agents and continue by following and answering the questions given by the CI.

Reflect on: 
* What do you think about the accuracy of the information?
* How can the CI aid you in your daily work?
* How migh the CI model evolve with advancements in AI. How can it be further refined or expanded to address diverse and emerging challenges?

## Chat with PDF using AskYourPDF plugin

Many companies find immense benefit in being able to talk to their database and extract information from internal PDF documents. The ability to interact with PDFs in a conversational manner can save time by quickly extracting needed information. This is especially useful in settings where there's a need to access specific information from large or multiple PDF documents without having to go through them manually​.

In this assignment, we will use the plugin, "AskYourPDF" to interact with PDF documents in a conversational manner.

The AskYourPDF plugin uses a search algorithm to match queries with the content of PDF documents and retrieve the most relevant information from the PDFs. It leverages the power of the LLM's language skills to create a seamless interaction. Besides PDF, the tool supports TXT, PPT, and PPTX file formats. 

Start by creating a New Chat.

1. Choose GPT-4 and enable the "Plugins" Beta. 
2. Enable AskYourPDF by ticking the box, as shown in the picture below.

![Image of GitClone](https://github.com/statisticalmodel/GenAI/blob/main/enable_plugins.png?raw=true)

Next step is to upload the document to AskYourPDF. Please be mindfull not to upload any secret information to this tool. In assignemnt 3 you will build your own tool to query pdfs. 

**Copy paste the following prompt into the ChatGPT window.**

In [6]:
"""
Provide the link where I can upload the document and help me summarize the pdf.
"""

'\nProvide the link where I can upload the document and help me summarize the pdf.\n'

Proceed by:
1. Clicking on "Upload Your Document"  
2. Click on "Click to upload", loacate the "Patricia_permobil_manual.pdf" in your Google Drive and double click on it.

![Image of GitClone](https://github.com/statisticalmodel/GenAI/blob/main/AskYourPDF1.png?raw=true)

A window saying "Analyzing document" appears and a "Document Id" is provided. Copy the "Document Id" and past it into the Chat GPT window. 

AskYourPDF proceeds by summarizing the content in the PDF


**Copy paste the following prompt into the ChatGPT window.**

In [8]:
prompt = f"""
What tools are needed for adjusting the Front Seat Height on Slipstream Single-Sided Forks
"""

Reflect on: 
* Where could this tool be usful?

Try changing the prompt for the above conversation in order to ask your own questions to the document.

## Analysis of Annual Reports via PDFs

In this assignment, we will analyze an annual report.

Follow the same proceedure as before and upload the PDF document named "Caterpillar-10k-short.pdf"

**Copy paste the following prompt into the ChatGPT window.**

In [9]:
prompt = f"""
Provide the link where I can upload the document and help me summarize the pdf.
"""

1. Clicking on "Upload Your Document"  
2. Click on "Click to upload", loacate the "Caterpillar-10k-short.pdf" in your Google Drive and double click on it.

Copy the "Document Id" and past it into the GPT window. 

AskYourPDF now provides a summary of the Annual report.

**Copy paste the following prompt into the ChatGPT window.**

In [10]:
prompt = f"""
Give me an executive summary of the company's financial condition and results of operations
"""

**Remember**

Always be mindful of security and never upload confidential documents to third-party tools. Always double check the correctness and verify the results. AI is a tool, not a replacement for human judgment. 

### Unmount Drive by running the below cell
> **NB: After completed course we can flush and unmount drive**

In [11]:
# First flush and unmount drive after we are done,
# but to re-mount with new login we may need to remove dir manually first
if use_drive:
  drive.flush_and_unmount()
  !rm -rf /content/drive
#drive.mount('/content/drive', force_remount=True)

NameError: name 'use_drive' is not defined